In [7]:
from pyvis.network import Network
import numpy as np
import random as rand

In [9]:
class Graph:
    """
    An undirected graph data structure storing uniquely indexed nodes with weighted edges

    Attributes:
        network: An instance of the Network class of the pyvis.network module
        node_map: A hashmap mapping the index of the node to its Node object 
        node_idx_count: An integer count for indexing new nodes
        num_nodes: The total number of nodes in the graph
        num_edges: The total number of edges in the graph
        starting_node: An integer index of the starting node defined by the user
        ending_node: An integer index of the ending node defined by the user

    Methods:
        generate_random_nodes:
        generate_random_edges:
        delete_edges:
        shortest_path:
        visualize_graph:
    """

    def __init__(self,
                 init_num_nodes: int = 0,
                 init_num_edges: int = 0) -> None:
        """
        Construct all attributes of the graph data structure.

        Args:
            init_num_nodes: Number of randomly generated nodes during initialization.
            init_num_edges: Number of randomly generated edges during initialization.

        Raises:
            ValueError: Errors caused by incorrect data type or invalid range inputs of 
                        parameters 'init_num_nodes' and 'init_num_edges'.
        """
        if not isinstance(init_num_nodes, int) or init_num_nodes < 0:
            raise ValueError("Input parameter 'init_num_nodes' must be a non-negative integer")
        if not isinstance(init_num_edges, int) or init_num_edges < 0:
            raise ValueError("Input parameter 'init_num_edges' must be a non-negative integer")

        self.network = Network()
        self.node_map = {}
        self.node_idx_count = 1
        self.num_nodes = 0
        self.num_edges = 0
        self.starting_node = None
        self.ending_node = None

        self.generate_random_edges(init_num_nodes)
        self.generate_random_edges(init_num_edges)

    def generate_random_nodes(self, num: int = 0) -> None:
        if not isinstance(num, int) or num < 0:
            raise ValueError("Input parameter 'num' must be a non-negative integer") 
        
        if num == 0:
            num = rand.randint(10, 15)
        self.num_nodes += num

        while num:
            self.__generate_node()
            num -= 1

    def generate_random_edges(self, num: int = 0) -> None:
        pass

    def delete_edges(self) -> None:
        pass

    def shortest_path(self) -> None:
        pass

    def visualize_graph(self) -> None:
        pass
    
    def __generate_node(self) -> None:
        pass